# Polinomial Regression

### Authors:
- Adrián Bedón
- Pablo Chasipanta
- José Miguel Merlo
- Dennis Ocaña
- Xavier Ramos

## 1. Polinomial Regression

In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt

In [3]:
# Load the dataset from the URL
url = "https://raw.githubusercontent.com/Machine1314/natural_computing/main/data_regression.csv"
df = pd.read_csv(url)

print(df.head())

# Extract the independent variable (X) and dependent variable (y)
X = df[['xi']].values 
y = df['yi'].values

   i    xi     yi
0  1   6.6  -45.4
1  2  10.1 -176.6
2  3   8.9 -127.1
3  4   6.0  -31.1
4  5  13.3 -366.6


# Create polynomial features (degree=2 for quadratic)
poly = PolynomialFeatures(degree=2)
X_poly = poly.fit_transform(X)

# Create and fit the linear regression model to the polynomial features
model = LinearRegression()
model.fit(X_poly, y)

# Make predictions
y_pred = model.predict(X_poly)

# Evaluate the model using metrics like MSE and R-squared
mse = mean_squared_error(y, y_pred)
r2 = r2_score(y, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")

# Sort the data points before plotting the predicted curve
sort_idx = np.argsort(X[:, 0])  # Get the indices to sort X
X_sorted = X[sort_idx]
y_pred_sorted = y_pred[sort_idx]

# Create a zoomed-out view
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))  # 1 row, 2 columns

# Original plot (ax1) - same as before
ax1.scatter(X, y, color='blue', label='Actual Data')
ax1.plot(X_sorted, y_pred_sorted, color='red', label='Predicted Curve')
ax1.set_xlabel('x')
ax1.set_ylabel('y')
ax1.set_title('Quadratic Regression: Original View')
ax1.legend()

# Zoomed-out plot (ax2)
x_range = np.linspace(X.min() - 5, X.max() + 5, 100)  # Extended x-range
X_range_poly = poly.transform(x_range.reshape(-1, 1))  # Transform for prediction
y_range_pred = model.predict(X_range_poly)

ax2.scatter(X, y, color='blue', label='Actual Data')
ax2.plot(x_range, y_range_pred, color='red', label='Predicted Curve')
ax2.set_xlabel('x')
ax2.set_ylabel('y')
ax2.set_title('Quadratic Regression: Zoomed-Out View')
ax2.legend()

plt.tight_layout()  # Adjust layout for better spacing
plt.show()

## 2. 

## 3. MLP

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt